# INST447: Data Sources and Manipulation
## Lecture 08: LLM APIs as Data Sources and Processors

Today's Topics:
- Data augmentation with AI-powered functions
- OpenAI and OpenRouter APIs
- Practical applications for text analysis
- Best practices and cost management

## Part 1: Motivation - Beyond Static Data Sources

### Recap: How We've Augmented Data So Far

**Method 1: Merging/Joining DataFrames**
- Combining data from multiple tables
- Example: Adding customer demographics to purchase data
- Limitation: Need pre-existing structured data

**Method 2: Applying Functions**
- Using .apply() or .map() to transform data
- Example: Converting temperatures, calculating totals
- Limitation: Functions must be pre-defined

**Method 3: Using APIs to Fetch Data (Lecture 07)**
- Retrieving external data via web APIs
- Example: Getting product info from Open Food Facts
- Limitation: Can only get what the API offers


### Today's Challenge: What If We Need Custom Analysis?

**Real-World Scenarios:**
- 📊 Analyzing sentiment of 10,000 customer reviews
- 🏷️ Classifying news articles into custom categories
- 🔍 Extracting named entities (people, places, companies) from text
- ✅ Verifying claims or fact-checking statements
- 🌍 Translating content while preserving context
- 📝 Summarizing long documents

**Traditional Approaches (and Their Problems):**

1. **Local Libraries (NLTK, spaCy, scikit-learn)**
   - ✅ Pro: Fast, no internet needed, free
   - ❌ Con: Limited capabilities, rigid, often need training data
   - ❌ Con: Poor with nuance, context, or novel tasks

2. **Custom Machine Learning Models**
   - ✅ Pro: Can be highly accurate for specific tasks
   - ❌ Con: Requires ML expertise, training data, and infrastructure
   - ❌ Con: Time-consuming to develop and maintain

3. **Rule-Based Systems**
   - ✅ Pro: Deterministic, explainable
   - ❌ Con: Brittle, don't handle edge cases or natural language variation

### The LLM Solution

**You already know ChatGPT can handle these tasks!**

You could:
1. Copy a review into ChatGPT
2. Type: "What's the sentiment of this review?"
3. Copy the response
4. Paste into your spreadsheet
5. Repeat 10,000 times... 😱

**The problem:** Manual, slow, not scalable, error-prone

**The solution:** LLM APIs let you automate this!
- Same AI models (GPT-4, Claude, etc.)
- But accessible via code
- Process thousands of items automatically
- Integrate directly with pandas DataFrames


### Today's Running Example

We'll work with customer restaurant reviews and learn to:
- Classify sentiment (positive/negative/neutral)
- Extract key topics and entities
- Identify specific issues (service, food, ambiance)
- Generate concise summaries

Let's start by setting up our sample dataset...

In [ ]:
# Setup: Import libraries
import pandas as pd
import json
import os
import time

# Create sample dataset - restaurant reviews
reviews_df = pd.DataFrame({
    'review_id': range(1, 11),
    'customer_name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve',
                     'Frank', 'Grace', 'Henry', 'Iris', 'Jack'],
    'review_text': [
        "Absolutely loved this place! The pasta was phenomenal and the service was impeccable.",
        "Worst experience ever. Cold food, rude staff, and overpriced. Never coming back.",
        "It was fine. Nothing special, nothing terrible. Just average Italian food.",
        "Amazing ambiance and the tiramisu was to die for! Will definitely return.",
        "Overpriced and underwhelming. The portions were tiny and lacked flavor.",
        "Good food but had to wait 45 minutes for a table despite having a reservation.",
        "Incredible! Best Italian food I've had outside of Italy. The chef is a genius.",
        "Mediocre at best. Expected more given all the positive reviews I read.",
        "The pizza was decent but our waiter was so attentive and friendly.",
        "Disappointing. The carbonara was bland and the restaurant was too noisy."
    ],
    'date': pd.date_range('2024-01-01', periods=10, freq='D')
})

print("Sample Restaurant Reviews Dataset:")
print(reviews_df)
print("\n" + "="*80 + "\n")

## Part 2: Understanding LLM APIs

### What is an LLM API?

**Recall from last week: Traditional APIs**
- Request specific, predefined data
- Example: "Give me product info for barcode X"
- Response: Structured data (JSON)
- **Function is predefined by the API provider**

Example from Lecture 07:
```
GET https://world.openfoodfacts.org/api/v0/product/3017620425035.json
→ Returns: Product data (name, nutrients, ingredients, etc.)
```

**LLM APIs: A Different Paradigm**
- Send a text prompt (instruction + data)
- LLM generates a response based on understanding
- Response: Generated text (may need parsing)
- **Function is defined by YOU in the prompt**

Example LLM API call:
```
POST https://api.openai.com/v1/chat/completions
Body: {
  "model": "gpt-3.5-turbo",
  "messages": [{"role": "user", "content": "Classify sentiment: The food was great!"}]
}
→ Returns: "positive"
```

**Key Difference:**
- Traditional API: A lot of endpoints, each corresponding to a predefined function.
- LLM API: Few "endpoitns", functions defined by prompts.

### Popular LLM API Providers

**Closed-Source Models:**

**1. OpenAI (GPT Models)**
- Models: GPT series (GPT5, GPT5-mini, GPT5-nano), o-series, etc.
- Industry standard, best documentation

**2. Anthropic (Claude)**
- Models: Claude Opus/Sonnet/Haiku
- Known for long context and safety

**3. Google (Gemini)**
- Models: Gemini 2.5 Pro, Flash
- Multimodal (text + images)


**Open-Source Models:**

**1. Meta (Llama)**
- Good performance, lighter weight

**2. DeepSeek**
- Strong reasoning capabilities, cost-effective

**3. Alibaba (Qwen)**
- Multilingual, good for Asian languages
- Multiple size to choose from

**Deployment Options:**

- **Cloud APIs:** OpenAI, Anthropic, Google (pay per use)
- **OpenRouter:** Gateway to 100+ models with one API key (includes free models!)
- **Local deployment:** Run Llama, DeepSeek, Qwen on your own hardware (free, private)

### How LLM APIs Work (Conceptual Flow)

1. **You prepare a prompt:**
   - Instruction: "Classify the sentiment of this review"
   - Data: "The food was amazing!"
   - Format request: "positive, negative, or neutral"

2. **Send to API:**
   - HTTP POST request with your prompt
   - Include API key for authentication
   - Specify model to use

3. **LLM processes:**
   - Model analyzes your prompt
   - Generates response based on training
   - Returns text output

4. **You receive and parse:**
   - Extract generated text from JSON response
   - Parse into desired format (may need cleaning)
   - Handle errors if response is unexpected

5. **You pay:**
   - Cost based on tokens (chunks of text)
   - Both input tokens (your prompt) and output tokens (response)
   - Example: ~$0.0005 for 1000 tokens on GPT-3.5

### Important Concepts

**Tokens:**
- Not the same as words
- Roughly: 1 token ≈ 4 characters ≈ 0.75 words
- Example: "Hello, world!" ≈ 4 tokens
- Both input and output count toward cost

**Models:**
- Different models = different capabilities, speed, and costs
- GPT5 -> GPT5-mini -> GPT5-nano
- Claude Opus -> Claude Sonnet -> Claude Haiku
- Choose based on task complexity

**Temperature (optional parameter):**
- Controls randomness (0 = deterministic, 2 = creative)
- For data processing or debugging, usually want low temperature (0-0.3)
- For creative tasks, higher temperature (0.7-1.5)
> What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic. (OpenAI documentation)

## Part 3: Making Your First LLM API Call

### Getting Set Up (Brief Overview)

1. **An API Key**
   - OpenAI: Sign up at https://platform.openai.com/, create API key
   - OpenRouter: Sign up at https://openrouter.ai/ (has FREE models!)
   - Format: `sk-proj-abc123...xyz789`
   - ⚠️ Keep it secret - treat it like a password!

2. **Security: Use Environment Variables**
   ❌ **NEVER:** `api_key = "sk-proj-abc123xyz"  # Hardcoded!`
   ✅ **DO:** `api_key = os.getenv('OPENAI_API_KEY')`

### Step 3.2: Understanding Message Roles - How Chat Actually Works

**LLM APIs use a "chat" format with different roles:**

**1. System Message (optional but powerful):**
   - Sets the behavior/personality of the assistant
   - Example: "You are a helpful data analyst who gives concise answers."
   - Think of it as "programming" the AI's behavior
   - Stays consistent across the conversation

**2. User Message:**
   - Your actual request or question
   - Example: "Classify the sentiment of this review: ..."
   - This is what you're asking the AI to do

**3. Assistant Message:**
   - The AI's responses
   - When building chat history, previous responses go here

**How chat history works:**

**Important:** Each API call is independent! The model doesn't "remember" previous calls.

To build conversation context, YOU must include previous messages:
```python
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What's 2+2?"},
    {"role": "assistant", "content": "4"},  # YOU add this from previous response
    {"role": "user", "content": "What about 2+3?"}  # Now AI has context!
]
```

**Key insight:**
- Each API call is stateless (no memory between calls)
- To maintain context, include previous messages in the array
- The model sees what you send, nothing more

**For data processing (our focus today):**
- We usually use simple system + user messages
- No history needed - each review is independent

In [2]:
# API Key setup
# In production: api_key = os.getenv('OPENAI_API_KEY')
# For demo, we'll use a burner key.
OPENAI_API_KEY = "your-api-key-here"  # Replace with your actual key

### Method 1 - Raw API Call with Requests

Let's start by making an API call the "hard way" using the `requests` library.
This helps us understand what's actually happening under the hood.

In [ ]:
import requests

def call_openai_raw(prompt_text, api_key):
    url = "https://api.openai.com/v1/chat/completions" # OpenAI Chat Completions endpoint

    # Request headers
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # Request body
    data = {
        "model": "gpt-5-mini",  # Fast, cheap, high quality (2025 standard)
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt_text}
        ],
    }

    # Make the request
    response = requests.post(url, headers=headers, json=data)

    # Check if successful
    if response.status_code == 200:
        result = response.json()
        return result
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

prompt = "What is the capital of France? Answer in one word."

raw_response = call_openai_raw(prompt, OPENAI_API_KEY)

Error: 401
{
    "error": {
        "message": "Incorrect API key provided: your-api*****here. You can find your API key at https://platform.openai.com/account/api-keys.",
        "type": "invalid_request_error",
        "param": null,
        "code": "invalid_api_key"
    }
}



In [ ]:
response

In [ ]:
print("Full API Response:")
print(json.dumps(raw_response, indent=2))

In [ ]:
print("Just the answer:")
answer = raw_response['choices'][0]['message']['content']
print(answer)

**What We See in the Response:**

The API returns a large JSON object with:
- `id`: Unique identifier for this completion
- `object`: Type of object ("chat.completion")
- `created`: Timestamp
- `model`: Which model was actually used
- `choices`: Array of generated responses (usually just one)
  - `message`: The actual response
    - `role`: "assistant"
    - `content`: **The text we want!**
  - `finish_reason`: Why it stopped ("stop" = completed naturally)
- `usage`: Token counts
  - `prompt_tokens`: Tokens in your prompt
  - `completion_tokens`: Tokens in the response
  - `total_tokens`: Sum (this determines cost!)

**To get the actual answer, we navigate:**
```python
answer = response['choices'][0]['message']['content']
```

### Method 2 - Using the OpenAI Wrapper Library

OpenAI provides an official wrapper that makes our lives much easier!

**Benefits of wrappers:**
- Cleaner code
- Better error handling
- Automatic retries
- Type hints and documentation
- Less manual JSON parsing

In [ ]:
from openai import OpenAI

def call_openai_wrapper(prompt_text, api_key):
    client = OpenAI(api_key=api_key)  # Initialize the client

    # Make the call
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt_text}
        ],
        temperature=0.3
    )

    # Extract the answer
    return response.choices[0].message.content

answer = call_openai_wrapper(prompt, OPENAI_API_KEY)
print(f"Question: {prompt}")
print(f"Answer: {answer}")

## Part 4: Applying LLM APIs to Real Data

### The Core Pattern: DataFrame + AI Analysis

**Remember from pandas:**
- We can add new columns with calculations: `df['total'] = df['price'] * df['quantity']`
- We can apply functions: `df['uppercase'] = df['name'].apply(str.upper)`

**Today, we'll do:**
```python
df['sentiment'] = df['review_text'].apply(analyze_with_llm)
```

Where `analyze_with_llm` sends each review to an LLM API!

### Single Review Analysis

Let's start by analyzing just ONE review to understand the process.

In [ ]:
import openai

def analyze_sentiment(text: str) -> dict:

    # Set up OpenAI API key
    openai.api_key = OPENAI_API_KEY

    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": "You are a sentiment analysis expert. Analyze the sentiment of the given text and respond with JSON containing: sentiment (positive/negative/neutral), score (0-1), and a brief explanation."
            },
            {
                "role": "user",
                "content": f"Analyze the sentiment of this text: {text}"
            }
        ]
    )

    result = eval(response.choices[0].message.content)
    return result


# Test on a single review
sample_review = reviews_df.iloc[0]['review_text']
sentiment = analyze_sentiment(sample_review)
print(f"Review: {sample_review}")
print(f"Sentiment: {sentiment}")

In [ ]:
sentiment['sentiment']

### Applying to Entire DataFrame

Now let's apply sentiment analysis to ALL reviews!

In [ ]:
# Note: In practice, this would make 10 API calls (one per review)
reviews_df['sentiment'] = reviews_df['review_text'].apply(analyze_sentiment)

print("Reviews with Sentiment Analysis:")
print(reviews_df[['customer_name', 'review_text', 'sentiment']])
print()

**What just happened?**

1. For each row in the DataFrame:
   - Extract the review_text
   - Call analyze_sentiment() with that text
   - LLM analyzes and returns sentiment
   - Result stored in new 'sentiment' column

2. In real usage with actual API:
   - This made 10 API calls (one per review)
   - Each call costs ~$0.0001 (tiny, but adds up!)
   - Total time: ~10-20 seconds (depending on API speed)

### Limitations of .apply() & Production Considerations

**The .apply() approach is great for learning, but has limitations for large-scale:**

- **Sequential processing:** One at a time (slow for thousands of rows)
- **No progress saving:** Crash at row 1500 of 2000 = start over
- **No error handling:** One API error crashes everything

**For production with large datasets, consider:**
- Batch processing with checkpointing (save progress periodically)
- OpenAI Batch API (50% cheaper, submit jobs and retrieve later)
- Async/concurrent requests (much faster)

**For this lecture:** We'll stick with .apply() for simplicity. Just know better tools exist!

### Step 4.4: Comparison with Traditional Methods

Before we move on, let's compare the LLM approach with a traditional library.

**Traditional approach: TextBlob (NLTK-based)**

In [ ]:
# Traditional sentiment analysis with TextBlob

from textblob import TextBlob

def sentiment_textblob(text):
    """Traditional sentiment analysis using TextBlob"""
    score = TextBlob(text).sentiment.polarity
    if score > 0.1:
        return "positive"
    elif score < -0.1:
        return "negative"
    return "neutral"

reviews_df['textblob_sentiment'] = reviews_df['review_text'].apply(sentiment_textblob)
print("Reviews with TextBlob Sentiment Analysis:")
print(reviews_df[['customer_name', 'review_text', 'textblob_sentiment']])

## Part 5: OpenRouter - Gateway to Many Models

**OpenRouter: One API for 100+ models**

Instead of signing up for OpenAI, Anthropic, Google separately:
- One API key for all providers
- Access to free models (Llama, DeepSeek, Qwen, etc.)
- Same code structure as OpenAI

**How to use:**
```python
from openai import OpenAI

# Just change the base_url!
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv('OPENROUTER_API_KEY')
)

response = client.chat.completions.create(
    model="meta-llama/llama-3.1-8b-instruct:free",  # Free model!
    messages=[{"role": "user", "content": "..."}]
)
```

**When to use OpenRouter:**
- ✅ Want to experiment with free models
- ✅ Compare different models easily
- ✅ Access models from multiple providers

## Part 6: Best Practices - Key Things to Remember

### Cost Management
- **Costs add up:** $0.0001 per review × 10,000 reviews = $1
- **Use cheaper models when possible:** GPT-5-mini vs GPT-5
- **Set spending limits** in API dashboard
- **Cache results:** Don't reprocess the same data

### Error Handling
- **APIs fail** (network issues, service outages like AWS)
- **Implement retries:** Try 2-3 times before giving up
- **Save progress frequently:** Don't lose work on crash
- **Log errors** for debugging

### Rate Limiting
- **APIs have limits** depending on tier
- **Add delays:** `time.sleep(1)` between requests
- **Or use batch APIs:** Submit many requests at once

### Security
- **Never commit API keys to git!**
- **Use environment variables:** `os.getenv('OPENAI_API_KEY')`
- **Set spending limits** in dashboard
- **Rotate keys** if exposed

### Prompt Engineering Basics
- **Be specific:** "Classify as positive, negative, or neutral" (not just "sentiment?")
- **Specify format:** "Return JSON with sentiment and confidence"
- **Test and iterate:** Try on examples, refine prompt